In [1]:
import requests
import pandas as pd
import json
from tqdm import tqdm 
from json.decoder import JSONDecodeError



In [2]:
# importando .csv com dados de nome e região de cada estado
info_ufs = pd.read_csv('dados_estados.csv')
info_ufs.head()

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


### Dados totais de nascimentos por estado

In [3]:
# para evitar erro 403 (sem autorização)
session = requests.Session()
session.headers.update({'User-Agent': 'Custom user agent'})
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}


# raspando dados de registros totais por uf
resposta_estados = session.get('https://transparencia.registrocivil.org.br/api/record/birth')
resposta_estados = resposta_estados.json()

In [4]:
resposta_estados['data']

[{'name': 'AC', 'total': 732133},
 {'name': 'AL', 'total': 1750882},
 {'name': 'AP', 'total': 459038},
 {'name': 'AM', 'total': 523642},
 {'name': 'BA', 'total': 7269226},
 {'name': 'CE', 'total': 5509069},
 {'name': 'DF', 'total': 1950182},
 {'name': 'ES', 'total': 3611459},
 {'name': 'GO', 'total': 2602510},
 {'name': 'MA', 'total': 1184852},
 {'name': 'MT', 'total': 685479},
 {'name': 'MS', 'total': 1054285},
 {'name': 'MG', 'total': 20609614},
 {'name': 'PA', 'total': 1248518},
 {'name': 'PB', 'total': 983199},
 {'name': 'PR', 'total': 12467213},
 {'name': 'PE', 'total': 12399812},
 {'name': 'PI', 'total': 394434},
 {'name': 'RJ', 'total': 4080974},
 {'name': 'RN', 'total': 772721},
 {'name': 'RS', 'total': 7642386},
 {'name': 'RO', 'total': 1030207},
 {'name': 'RR', 'total': 187977},
 {'name': 'SC', 'total': 4837339},
 {'name': 'SP', 'total': 50522961},
 {'name': 'SE', 'total': 1253404},
 {'name': 'TO', 'total': 474727}]

### rapando dados do ranking dos 50 principais nomes por estado

In [ ]:
estados = []
df_estados = pd.DataFrame()
for uf in info_ufs.values: #resposta_estados['data']
    
    uf_nome = uf[1]
    resposta = session.get(f'https://transparencia.registrocivil.org.br/api/record/all-name?start_date=2022-01-01&end_date=2022-12-31&translate=1&state={uf_nome}')
    print(f"{uf[1]}: {resposta.status_code}")
    resposta = resposta.json()
    df_resposta = pd.DataFrame(resposta['data'])
    df_resposta['uf'] = uf_nome
    df_resposta['regiao'] = uf[5]
    df_resposta['ano'] = 2021
    df_resposta.index = df_resposta.index + 1
    
    df_estados = df_estados.append(df_resposta)
    
    print('---')

In [6]:
df_estados.head()

,total,name,uf,regiao,ano
1,314,MIGUEL,RO,Norte,2021
2,241,GAEL,RO,Norte,2021
3,240,ARTHUR,RO,Norte,2021
4,235,HEITOR,RO,Norte,2021
5,233,HELENA,RO,Norte,2021


In [8]:
df_estados.to_csv('nomes_mais_registrados_br_2022.csv')

### Dados por município

In [9]:
resposta_municipios = session.get('https://transparencia.registrocivil.org.br/api/cities')
resposta_municipios = resposta_municipios.json()

In [10]:
resposta_municipios['cities'][0:11]

[{'uf': 'SP', 'name': 'São Paulo', 'id': 1},
 {'uf': 'SP', 'name': 'São Bernardo do Campo', 'id': 2},
 {'uf': 'SP', 'name': 'Sumare', 'id': 3},
 {'uf': 'SP', 'name': 'São Caetano do Sul', 'id': 4},
 {'uf': 'SP', 'name': 'Santo Andre', 'id': 5},
 {'uf': 'SP', 'name': 'Barueri', 'id': 6},
 {'uf': 'SP', 'name': 'Ribeirão Pires', 'id': 7},
 {'uf': 'SP', 'name': 'Campinas', 'id': 8},
 {'uf': 'SP', 'name': 'Paulinia', 'id': 9},
 {'uf': 'SP', 'name': 'Salto', 'id': 10},
 {'uf': 'SP', 'name': 'Cotia', 'id': 11}]

#### Tranformando nome dos municípios em no formato URL encode

In [11]:
# testanto biblioteca
import urllib.parse
query = 'Jequiá da Praia'
urllib.parse.quote(query)


'Jequi%C3%A1%20da%20Praia'

### rapando dados dos municípios brasileiros

In [12]:

df_municipios = pd.DataFrame()

for item in tqdm(resposta_municipios['cities']):
    try:
        municipio_encode = urllib.parse.quote(item['name'])        
        uf_busca = item['uf']     
        resposta = session.get(f'https://transparencia.registrocivil.org.br/api/record/all-name?start_date=2022-01-01&end_date=2022-12-31&translate=1&state={uf_busca}&city={municipio_encode}')

        resposta = resposta.json()
        df_resposta = pd.DataFrame(resposta['data'])
        df_resposta['uf'] = uf_busca
        df_resposta['municipio'] = item['name']
        df_resposta['ano'] = 2022
        df_resposta.index = df_resposta.index + 1
        #display(df_resposta)
        df_municipios = df_municipios.append(df_resposta)

    except json.decoder.JSONDecodeError:
        print(item['name']) 
        print(item['uf'])
        print(resposta.status_code)
        print("String could not be converted to JSON")
        continue  
         



  0%|          | 0/5611 [00:00<?, ?it/s]/var/folders/6x/mpdqd10961j55x6_pl8spyqmvpkspx/T/ipykernel_18524/1365314433.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_municipios = df_municipios.append(df_resposta)
  0%|          | 1/5611 [00:00<1:17:42,  1.20it/s]/var/folders/6x/mpdqd10961j55x6_pl8spyqmvpkspx/T/ipykernel_18524/1365314433.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_municipios = df_municipios.append(df_resposta)
  0%|          | 2/5611 [00:01<1:15:31,  1.24it/s]/var/folders/6x/mpdqd10961j55x6_pl8spyqmvpkspx/T/ipykernel_18524/1365314433.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_municipios = df_municipios.append(df_resposta)
  0%|          | 3/5611 [00:02<1:14:04,  1.26it/s]/var/f

In [13]:
#total de municípios com dados coletados
len(df_municipios.groupby('municipio').sum())

4899

In [14]:
df_municipios.to_csv('nomes_mais_registrados_municipios_2022.csv')